In [1]:
import os
import cv2
import matplotlib.pyplot as plt
# seed the pseudorandom number generator
from random import seed
from random import random
from random import randint
import numpy as np
seed(0)

In [2]:
sdr_folder = 'data/sdr/'
orig_folder = 'data/original/'
dest_sdr_folder = 'data_augmented/sdr/'
dest_orig_folder = 'data_augmented/original/'


image_paths = []
for root, dirs, files in os.walk(sdr_folder, topdown=False):
    for name in files:
        image_paths.append(os.path.join(root, name))
    #for name in dirs:
     #   print(os.path.join(root, name))
dataset_size = len(image_paths)
print(dataset_size)

32400


In [3]:
def add_random_gaussian_blur(image):
    k_size = 2*randint(0,1)+1
    kernel = (k_size, k_size)
    dst = cv2.GaussianBlur(image,kernel,cv2.BORDER_DEFAULT)
    return dst
    
def add_random_median_blur(image):
    k_size = randint(0,3)
    if (k_size % 2) == 0:
        return image
    median = cv2.medianBlur(image,k_size)
    return median
def add_random_noise(image):
    row,col,ch = image.shape
    s_vs_p = 0.5
    amount = 0.004
    out = np.copy(image)
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
    out[coords] = 255

    # Pepper mode
    num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
    out[coords] = 0
    return out
def add_random_contrast(image):
    if(randint(0,1) == 0):
        return image
    lab= cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clip_limit = random()*2.0 + 1.0
    tile_grid_size = randint(6,9)
    
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(tile_grid_size,tile_grid_size))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final

def add_random_brightness(image):
    img = image.copy()
    brightness_factor = 0.8 + random()*0.3
    img = np.float32(img)/255.0
    img *= brightness_factor
    img = np.clip(img,0.0,1.0)
    out = np.uint8(img*255)
    return out
def add_random_color_inversion(image):
    if(randint(0,1) == 0):
        return image
    return(np.uint8(255-image))

In [5]:
k = 1
for idx in range(1, dataset_size+1):
    sdr_img = cv2.imread(sdr_folder+str(idx)+".png", 1)
    orig_image = cv2.imread(orig_folder+str(idx)+".png", 1)
    cv2.imwrite(dest_orig_folder + str(k) + ".png", orig_image)
    cv2.imwrite(dest_sdr_folder + str(k) + ".png", sdr_img)
    k += 1
    aug = add_random_contrast(sdr_img)
    aug = add_random_brightness(aug)
    aug = add_random_gaussian_blur(aug)
    aug = add_random_median_blur(aug)
    cv2.imwrite(dest_orig_folder + str(k) + ".png", orig_image)
    cv2.imwrite(dest_sdr_folder + str(k) + ".png", aug)
    k+=1
print("{} images stored".format(k))

64801 images stored


In [6]:
image_paths = []
for root, dirs, files in os.walk(dest_sdr_folder, topdown=False):
    for name in files:
        image_paths.append(os.path.join(root, name))
    #for name in dirs:
     #   print(os.path.join(root, name))
dataset_size = len(image_paths)
print(dataset_size)

64800
